# Untanglement

In this notebook, we show how you can use ContourCraft to untangle several garments. That is, gives a set of potentially intersecting garments in canonical poses, untangle their geomenties to remove all visible penetrations.

![untanglement](static/untangle.png)

First, for each garment, you'll need to creata a .pkl garment dict file. Please refer to the section *"Add your own garment from an `.obj` file"* in [GarmentImport.ipynb](GarmentImport.ipynb) for the instructions on how to do that.

Then, create an `Untangler` object. It will be performing the simulations to untangle the garments.

Set the path to the ContourCraft model checkpoint and the directory containing the garment dict files. Here we use example values that should work if you have followed the [data preparation instructions](INSTALL.md#download-data) correctly.

The simulation untangles garments one by one from the innermost to the outermost with the loop which performs the following two-stage algorithm for several epochs:
1. Simulate all inner garments as rigid bodies and the outermost garment as cloth to push it outside using the penetration loss.
2. Simulate all the garments together as cloth to relax the inner garments and resolve remaining penetration with the Intersection Contour loss.

You may need to adjust the parameters `n_epochs` and `n_steps_per_stage` to achieve satisfying results for your case.

In [ ]:
from utils.untanglement import Untangler
from pathlib import Path
from utils.defaults import DEFAULTS
from utils.mesh_creation import GarmentCreator

In [ ]:
models_dir = Path(DEFAULTS.data_root) / 'trained_models'
checkpoint_path = models_dir / 'contourcraft.pth'
body_models_root = Path(DEFAULTS.aux_data) / 'body_models'

# Model type, either 'smpl' or 'smplx'
model_type = 'smplx'
# gender, either 'male`, `female` or `neutral`
gender = 'male'

garment_dicts_dir = Path(DEFAULTS.aux_data) / 'garment_dicts' / model_type

n_epochs = 2 # Number of simulation epochs for each outer garment 
n_steps_per_stage = 30 # Number of simulation steps per stage

garment_creator = GarmentCreator(garment_dicts_dir, body_models_root, model_type, gender)
untangler = Untangler(garment_creator, checkpoint_path, n_steps_per_stage=n_steps_per_stage, n_epochs=n_epochs, gender=gender, use_uv=False, garment_dicts_dir=garment_dicts_dir)

Now you can run the untanglement simulation. Add the names of your garments to the `garment_list_in_order` in the order from the innermost to the outermost. The garment names correspond to the garment dict file paths relative to the `garment_dicts_dir` you defined above.

After the untanglement is finished the updated garment dicts will be stored in the outfit folder you define in the `outfit_name`.

Then, you'll be able to simulate the untangled garments in new poses by setting the `garment_name` parameter (e.g. in [Inference.ipynb](Inference.ipynb)) either to the name of the outfit
```python
garment_name = "{outfit_name}"
```
or to a comma separated list of the garment names preceded by the outfit name:
```python
garment_name = "{outfit_name}/{garment_name1},{outfit_name}/{garment_name2},..."
```

In [ ]:
garment_list_in_order = []

garment_list_in_order.append('aaron_022::top_blouse')
garment_list_in_order.append('ben_004::top_tshirt')
garment_list_in_order.append('aaron_022::top_coat')

outfit_name = 'combined_test'

untanglement, trajectory_dict = untangler.untangle_all(garment_list_in_order, outfit_name)

For debugging purposes, you may see the unganglement simulation in AITViewer.

To do that,
1. Save the untanglement trajectory to a .pkl file (see below)
2. Start AITViewer with `python utils/show.py rollout_path=PATH_TO_SEQUENCE`

In [ ]:
from utils.io import pickle_dump


out_path = Path(DEFAULTS.data_root) / 'temp' / 'untanglement_debug.pkl'
out_path.parent.mkdir(parents=True, exist_ok=True)
pickle_dump(trajectory_dict, out_path)